In [1]:
import os
import json
import google.generativeai as genai
import re


# Replace with your actual API key (don't include here)
GOOGLE_API_KEY = "API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)


def list_genai_models():
  """Lists available GenerativeAI models (optional)."""
  for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
      print(m.name)


def read_file_as_string(filename):
  """Reads a file and returns its contents as a string."""
  with open(filename, "r") as f:
    content = f.read()
  return content


def generate_content(model, prompt):
  """Generates text using the provided model and prompt."""
  response = model.generate_content(prompt)
  return response.text

def sanitize_file_name(file_name):
    # Remove special characters except for spaces and parentheses
    sanitized_name = re.sub(r'[^\w\s()]', '', file_name)
    # Replace spaces with underscores
    sanitized_name = sanitized_name.replace(' ', '_')
    return sanitized_name


def write_output_to_json(data, filename):

  """Writes a dictionary or string to a JSON file."""
  with open(filename, "w") as f:


    start_ind = data.find('{')
    end_ind = data.rfind('}')


    substr = data[start_ind :end_ind+1]
    if start_ind != -1 and end_ind != -1:
      try:
        data = json.loads(substr.replace("\\n", "\n").replace('\\','').strip())
        json.dump(data, f, indent=2, ensure_ascii=False)
      except json.JSONDecodeError:
       print("JSON decoding error occurred. Saving the file using a dictionary instead.")
       json_data = {"task_tree": data.replace("\\n", "\n").replace('\\','').strip()}
       json.dump(json_data, f, indent=2, ensure_ascii=False)
    else:

      json.dump(dict(task_tree = data.replace("\\n", "\n").replace('\\','').strip()),f,ensure_ascii= False, indent =2)
  """Writes a dictionary or string to a JSON file."""
  # with open(filename, "w") as f:


  #   start_ind = data.find('{')
  #   end_ind = data.rfind('}')


  #   substr = data[start_ind :end_ind+1]

  #   data = json.loads(substr.replace("\\n", "\n").replace('\\','').strip())
  #   json.dump(data, f, indent=2, ensure_ascii=False)





def extract_kitchen_items(kitchen_text):

  items = []
  items.extend([word.lower() for word in kitchen_text.split() if word.isalpha()])
  return list(set(items))  # Remove duplicates


if __name__ == "__main__":

  # Load the model
  model = genai.GenerativeModel("gemini-1.0-pro-latest")

  # Read input data from JSON file
  with open("input.json", "r") as f:
    data = json.load(f)

  # Iterate through each category (dictionary) in the JSON list
  for category in data:
    # Access the "menu" list within the current category dictionary
    menu = category["menu"]

    # Iterate through each dish (dictionary) within the "menu" list
    for dish in menu:
      dish_name = dish["dish"]

      # Read the prompt text from prompt.txt
      prompt_text = read_file_as_string("prompt.txt")

      # Read kitchen instructions 
      kitchen_text = read_file_as_string("kitchen.txt") if os.path.exists("kitchen.txt") else ""

      # Extract kitchen items from kitchen.txt
      kitchen_items = extract_kitchen_items(kitchen_text)

      # Extract ingredients from dish description
      ingredients = [word.lower() for word in dish["dish"].split() if word.isalpha()]

      # Check kitchen inventory based on kitchen.txt
      can_make_dish = all(ingredient in kitchen_items for ingredient in ingredients)

      prompt = f"""From this prompt: {prompt_text}

Considering dish: {dish_name}

Using kitchen items:
{kitchen_text}

**Goal:**  if {can_make_dish} else 'Unfortunately, it looks like you don\'t have all the ingredients to make {dish_name}. Generate a detailed task tree for cooking {dish_name}.'

Please refer to the provided examples and structure within the prompt text for the desired format of the task tree.

Use your knowledge and creativity to generate a similar task tree for {dish_name}.
"""

      # Generate task tree using model
      task_tree = generate_content(model, prompt)

      #print(task_tree)

      # Store results in the filename with dish name
      write_output_to_json(task_tree, f"output_{sanitize_file_name(dish_name)}.json")

      # Print success message with dish name
      print(f"Task tree for {dish_name} generated and saved to output_{dish_name}.json")


Task tree for Latin Beef Stew generated and saved to output_Latin Beef Stew.json
Task tree for Hearty Mashed Potatoes generated and saved to output_Hearty Mashed Potatoes.json
Task tree for Mushroom & Swiss Burger generated and saved to output_Mushroom & Swiss Burger.json
Task tree for Char-Grilled Turkey Burger generated and saved to output_Char-Grilled Turkey Burger.json
Task tree for Grilled Black Bean Burger generated and saved to output_Grilled Black Bean Burger.json
Task tree for Hamburger generated and saved to output_Hamburger.json
Task tree for Dill Pickle Slices generated and saved to output_Dill Pickle Slices.json
Task tree for Crispy Chicken Sandwich generated and saved to output_Crispy Chicken Sandwich.json
Task tree for Crispy Shoestring French Fries generated and saved to output_Crispy Shoestring French Fries.json
Task tree for Kielbasa & Potato Hash generated and saved to output_Kielbasa & Potato Hash.json
Task tree for Sweet Tomato Soup (6 fl oz) generated and saved to